![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
# Face Recognition & Tracking Tutorials
This notebook has two tutorials:
1. How to perform face recognition tasks step by step
2. How to perform real-time face tracking

Prerequisites (uncomment to execute):

In [ ]:
# install degirum-face package
# %pip install degirum_face

# create cloud API access token
# !degirum token create

### Decide where your hardware is located
1. @cloud: To run on hardware hosted in DeGirum AI Hub
1. @local: To run on you current host
1. <host_ip>: To run on an AI server either on the same machine ('localhost') or in local LAN


In [ ]:
inference_host_address = "@cloud"  # hardware location

### Explore available hardware options

In [ ]:
import degirum_face

# What's in the registry?
registry_hw = degirum_face.model_registry.get_hardware()
print(f"Registry has {len(registry_hw)} hardware types:")
print(registry_hw)

# What's available on inference_host_address?
available_hw = degirum_face.get_system_hw(inference_host_address)
print(f"\n{inference_host_address} has {len(available_hw)} hardware types:")
print(available_hw)

# What can I actually use? (intersection)
compatible_hw = degirum_face.get_compatible_hw(inference_host_address)
print(f"\nYou can use {len(compatible_hw)} hardware types:")
print(compatible_hw)

### Decide which hardware to use
Choose one from `compatible_hw` printed just above

In [ ]:
hardware_to_use = "N2X/ORCA1"  # hardware to use for model inference. Choose one from compatible_hw above

### Face Recognition Tutorial

Define all necessary objects:
- face embeddings database
- face recognition configuration
- create face recognition object

In [ ]:
import degirum_face, degirum_tools, numpy as np

# define face recognition configuration
face_recognition_config = degirum_face.FaceRecognizerConfig(
    face_detection_model_spec=degirum_face.get_face_detection_model_spec(
        hardware_to_use, inference_host_address=inference_host_address
    ),
    face_embedding_model_spec=degirum_face.get_face_embedding_model_spec(
        hardware_to_use, inference_host_address=inference_host_address
    ),
    db_path="temp/tutorial_db.lance",
)

# create FaceRecognizer instance: this is the main object to use for face recognition
face_recognizer = degirum_face.FaceRecognizer(face_recognition_config)

Define helper function to display images and videos in Jupyter notebooks
(this is just a set of service functions for convenience)

In [ ]:
from IPython.display import Image, Video, display
import cv2, tempfile


def image_display(img):
    display(Image(data=cv2.imencode(".png", img)[1].tobytes()))


def video_display(video_bytes):
    with tempfile.TemporaryDirectory() as tmpdir:
        video_path = f"{tmpdir}/video.mp4"
        with open(video_path, "wb") as f:
            f.write(video_bytes)

        display(Video(filename=video_path, width=640, height=480, embed=True))

Enroll persons in the database by analyzing their face images to extract and store face embeddings

In [ ]:
# clear all tables to start fresh
face_recognizer.db.clear_all_tables()

# enroll Alice and Bob
enrolled = face_recognizer.enroll_batch(
    ("assets/Alice-1.png", "assets/Bob-1.png"), ("Alice", "Bob")
)

# print enrolled face information
print("Frame Name    DB ID")
for face in enrolled:
    print(f"{face.frame_id:5} {face.attributes:7} {face.db_id}")

# show number of enrolled embeddings in database
print("\n", face_recognizer.db.count_embeddings())

enrolled_embeddings = [face.embeddings[0] for face in enrolled]

Recognize enrolled persons using their other images

In [ ]:
# loop over other images
recognized_embeddings = []
for result in face_recognizer.predict_batch(
    (
        "assets/Alice-2.png",
        "assets/Alice-3.png",
        "assets/Bob-2.png",
        "assets/Bob-3.png",
        "assets/Alice&Bob.png",
    )
):
    # print recognition results
    print(f"\nImage '{result.info}'---------------")
    for n, face in enumerate(result.faces):
        print(f"Face #{n}:\n{face}")
        recognized_embeddings.append(face.embeddings[0])

    # display image overlay
    image_display(result.image_overlay)

Compute pairwise cosine similarities over face embeddings (just for demo purposes)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between every pair (recognized_embeddings, enrolled_embeddings)
sim_matrix = cosine_similarity(np.array(recognized_embeddings), np.array(enrolled_embeddings))
print("Cosine similarity matrix\n [Alice      Bob       ]")
print(sim_matrix)

### Face Tracking Tutorial

Define face tracking configuration

In [ ]:
# define configuration for face filtering
face_filters = degirum_face.FaceFilterConfig(
    enable_small_face_filter=True,
    min_face_size=30,
    enable_zone_filter=True,
    zone=[[100, 10], [960, 10], [960, 700], [100, 700]],
    enable_frontal_filter=True,
    enable_shift_filter=True,
    enable_reid_expiration_filter=True,
    reid_expiration_frames=12,
)

# define configuration for clip storage (use local directory for tutorial, but can be any S3-compatible storage)
clip_storage_config = degirum_tools.ObjectStorageConfig(
    endpoint="./temp", access_key="", secret_key="", bucket="tutorial_videos"
)

# define face tracking configuration
face_tracking_config = degirum_face.FaceTrackerConfig(
    video_source="assets/WalkingPeople.mp4",
    face_detection_model_spec=degirum_face.get_face_detection_model_spec(
        hardware_to_use, inference_host_address=inference_host_address
    ),
    face_embedding_model_spec=degirum_face.get_face_embedding_model_spec(
        hardware_to_use, inference_host_address=inference_host_address
    ),
    db_path="temp/tutorial_db.lance",
    face_filters=face_filters,
    clip_storage_config=clip_storage_config,
    clip_duration=300,
    alert_mode=degirum_face.AlertMode.ON_UNKNOWNS,
    credence_count=4,
)

# create clip manager instance
clip_manager = degirum_face.FaceClipManager(clip_storage_config)

# create FaceTracker instance
face_tracker = degirum_face.FaceTracker(face_tracking_config)

Run pipeline on database. We will collect video clips of unknown persons.

In [ ]:
# clear all saved clips first
clip_manager.remove_all_clips()

# then run the face tracking pipeline
composition, _ = face_tracker.start_face_tracking_pipeline()
composition.wait()

Analyze collected video clips

In [ ]:
# list all collected clips
all_clips = clip_manager.list_clips()
print(f"Clips saved: {len(all_clips)}")

# show them by downloading from storage:
for clip in all_clips:
    video_display(clip_manager.download_file(clip + ".mp4"))

Annotate collected video clips

In [ ]:
for clip in all_clips:
    # annotate clip: it will return face map object indexed by object track IDs and containing face embeddings
    face_map = face_tracker.find_faces_in_clip(clip)

    # show annotated clip
    video_display(
        clip_manager.download_file(clip + clip_manager.annotated_video_suffix)
    )

    break  # annotate only first clip

Add (enroll) all persons detected on the video clip to the database

In [ ]:
# You can see on a video above that object #1 is Bob and object #2 is Alice

# assign names to face objects
face_map[1].attributes = "Bob"
face_map[2].attributes = "Alice"

# enroll faces into the database
face_tracker.enroll(face_map.values())

# show number of enrolled embeddings in database
print(face_tracker.db.count_embeddings())

Run pipeline one more time: now should be no alerts

In [ ]:
# clear all saved clips first
clip_manager.remove_all_clips()

# start face tracking pipeline
composition, _ = face_tracker.start_face_tracking_pipeline()
composition.wait()

clips = clip_manager.list_clips()
print(f"Now we have: {len(clips)} clip(s)")
if not clips:
    print("No alerts detected!")